In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pydotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
import google.generativeai as genai
import os
import pandas as pd
import tiktoken
import numpy as np
genai.configure(api_key=os.environ["API_KEY"])

In [5]:
def get_embeddings( title, article ):
    result = genai.embed_content(
    model="models/text-embedding-004",
    content=article,
    task_type="retrieval_document",
    title=title) 
    return result['embedding']

In [6]:
df = pd.read_csv('./data/gdpr_cased_articles_with_recitals.csv')

In [7]:
df.head()

,article_id,article_title,article_text,article_recitals
0,article1,Subject-matter and objectives,This Regulation lays down rules relating to th...,"1,2,3,4,5,6,7,8,9,10,11,12,13"
1,article2,Material scope,This Regulation applies to the processing of p...,"14,15"
2,article2,Material scope,This Regulation does not apply to the processi...,16
3,article2,Material scope,This Regulation does not apply to the processi...,NaN
4,article2,Material scope,This Regulation does not apply to the processi...,18


In [8]:
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [9]:
list = []
for i in range(len(df.index)):
    title, text = df['article_title'][i], df['article_text'][i]
    token_len = num_tokens_from_string(text)
    temp = [title, text, token_len]
    embedding = get_embeddings(title, text)
    temp.append(embedding)
    list.append(temp)

In [10]:
df_new = pd.DataFrame(list, columns=['article_title', 'article_text', 'article_tokens', 'article_text_embeddings'])

In [11]:
df_new.head()

,article_title,article_text,article_tokens,article_text_embeddings
0,Subject-matter and objectives,This Regulation lays down rules relating to th...,86,"[-0.07772678, 0.0103528565, 0.0082582515, -0.0..."
1,Material scope,This Regulation applies to the processing of p...,45,"[-0.05222351, 0.053586897, -0.03747398, -0.035..."
2,Material scope,This Regulation does not apply to the processi...,27,"[-0.040194023, 0.017073363, -0.057278626, -0.0..."
3,Material scope,This Regulation does not apply to the processi...,37,"[-0.043380607, 0.019432526, -0.039794073, -0.0..."
4,Material scope,This Regulation does not apply to the processi...,27,"[-0.057984274, 0.0064719967, -0.05763235, -0.0..."


In [12]:
x = pd.merge(df, df_new, how="left")

In [13]:
x.head()

,article_id,article_title,article_text,article_recitals,article_tokens,article_text_embeddings
0,article1,Subject-matter and objectives,This Regulation lays down rules relating to th...,"1,2,3,4,5,6,7,8,9,10,11,12,13",86,"[-0.07772678, 0.0103528565, 0.0082582515, -0.0..."
1,article2,Material scope,This Regulation applies to the processing of p...,"14,15",45,"[-0.05222351, 0.053586897, -0.03747398, -0.035..."
2,article2,Material scope,This Regulation does not apply to the processi...,16,27,"[-0.040194023, 0.017073363, -0.057278626, -0.0..."
3,article2,Material scope,This Regulation does not apply to the processi...,NaN,37,"[-0.043380607, 0.019432526, -0.039794073, -0.0..."
4,article2,Material scope,This Regulation does not apply to the processi...,18,27,"[-0.057984274, 0.0064719967, -0.05763235, -0.0..."


In [14]:
x.to_csv('./data/gdpr_cased_articles_with_recitals_embeddings.csv')

In [15]:
def get_dot_product(text_embedding, question_embedding=""):
    return np.dot(text_embedding, question_embedding)

In [16]:
question_embedding = get_embeddings("user input", "What are my rights as an Individual in EU")

In [17]:
x['distance'] = x.article_text_embeddings.apply(lambda p: get_dot_product(p, question_embedding))

In [18]:
x.sort_values(['distance'], ascending=False)

,article_id,article_title,article_text,article_recitals,article_tokens,article_text_embeddings,distance
586,article77,Right to lodge a complaint with a supervisory ...,Without prejudice to any other administrative ...,141,70,"[-0.04318639, 0.009823787, 0.0013123561, -0.02...",0.760135
164,article22,"Automated individual decision-making, includin...",The data subject shall have the right not to b...,"71,72",39,"[-0.04324376, 0.015624178, 0.016327191, -0.046...",0.746726
588,article78,Right to an effective judicial remedy against ...,Without prejudice to any other administrative ...,143,40,"[-0.045312826, -0.0052126944, 0.026995184, -0....",0.735505
130,article15,Right of access by the data subject,The data subject shall have the right to obtai...,"63,64",58,"[-0.047044102, 0.01906889, 0.0070038955, -0.04...",0.733498
366,article47,Binding corporate rules,The binding corporate rules shall specify at l...,110,98,"[-0.04559353, 0.005038787, 0.028027346, -0.033...",0.725485
...,...,...,...,...,...,...,...
31,article4,Definitions,'group of undertakings' means a controlling un...,37,18,"[-0.026804209, 0.029797085, 0.0009110155, -0.0...",0.488253
598,article81,Suspension of proceedings,Where those proceedings are pending at first i...,144,51,"[0.016459905, 0.02905598, 0.016495313, -0.0320...",0.485155
567,article73,Chair,The term of office of the Chair and of the dep...,NaN,21,"[0.018717434, 0.05230803, 0.0047608837, -0.004...",0.479319
576,article75,Secretariat,"The secretariat shall provide analytical, admi...",140,15,"[0.036584362, 0.042329982, 0.016415294, 0.0004...",0.477369


In [19]:
x.loc[x["article_title"] == "Right to erasure ('right to be forgotten')"]

,article_id,article_title,article_text,article_recitals,article_tokens,article_text_embeddings,distance
136,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",68,"[-0.032365043, -0.008397134, 0.0020770333, -0....",0.695727
137,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",158,"[-0.033225175, -0.0029193629, -0.0044079283, -...",0.687852
138,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",89,"[-0.0455578, -0.013683858, 0.0030171564, -0.02...",0.706607
139,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",56,"[-0.042438716, -0.00021942731, -0.0029840148, ...",0.710405
140,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",73,"[-0.03386438, -0.0027421627, 0.002576739, -0.0...",0.713335
141,article17,Right to erasure ('right to be forgotten'),The data subject shall have the right to obtai...,"65,66",71,"[-0.05192303, -0.0018970313, -0.013146247, -0....",0.711652
142,article17,Right to erasure ('right to be forgotten'),Where the controller has made the personal dat...,"65,66",77,"[-0.039192215, 0.01568776, -0.027002933, -0.03...",0.691379
143,article17,Right to erasure ('right to be forgotten'),Right to erasure ('right to be forgotten') sha...,"65,66",32,"[-0.064188726, -0.017257614, -0.035147376, -0....",0.696933
144,article17,Right to erasure ('right to be forgotten'),Right to erasure ('right to be forgotten') sha...,"65,66",67,"[-0.04173252, 0.0028624088, -0.00333421, -0.01...",0.686422
145,article17,Right to erasure ('right to be forgotten'),Right to erasure ('right to be forgotten') sha...,"65,66",134,"[-0.028719774, 0.004937607, -0.025912542, -0.0...",0.670468


In [20]:
len(x["article_title"].unique())

97

In [21]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    langchain-google-community \
    google-cloud-discoveryengine \
    google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [22]:
google_storage_bucket="vertex-ai-hack"
google_storage_bucket_link="gs://{}".format(google_storage_bucket)
data_bucket="{}/data/".format(google_storage_bucket_link)
project_id="hackhathon-438922"

In [23]:
import vertexai
from vertexai.preview import reasoning_engines, rag
from vertexai.preview.generative_models import GenerativeModel, Tool
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('./keys/hackhathon-438922-0a5870e658d5.json')

vertexai.init(
    project=project_id,
    location="us-central1",
    staging_bucket="gs://{}".format(google_storage_bucket),
    credentials=credentials
)

In [24]:
display_name = "eu_ai_act"
paths = [data_bucket]

In [25]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client(project=project_id, credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )


In [26]:
model = "gemini-1.5-flash-001"

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}
model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 8192,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens. This is not supported by all model versions. See
    # https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/image-understanding#valid_parameter_values
    # for details.
    "top_k": None,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    # (you must create your safety settings using the previous step first).
    "safety_settings": safety_settings,
    "credentials": credentials
}

In [27]:
agent = reasoning_engines.LangchainAgent(
    model=model,                # Required.
    model_kwargs=model_kwargs
)

response = agent.query(input="Tell me about the EU AI act article")

In [28]:
response

{'input': 'Tell me about the EU AI act article',
 'output': "The EU AI Act is a proposed regulation by the European Union that aims to regulate the development, deployment, and use of artificial intelligence (AI) systems within the EU. It's a landmark piece of legislation that seeks to address the ethical, legal, and societal implications of AI.\n\n**Key Features of the EU AI Act:**\n\n* **Risk-Based Approach:** The Act categorizes AI systems into four risk levels: unacceptable risk, high risk, limited risk, and minimal risk. The level of regulation applied to each system depends on its potential impact.\n* **Prohibition of Unacceptable Risk AI:** The Act prohibits the use of AI systems that pose an unacceptable risk, such as those that manipulate human behavior, exploit vulnerabilities, or violate fundamental rights.\n* **Strict Requirements for High-Risk AI:** High-risk AI systems, such as those used in critical infrastructure, healthcare, and law enforcement, will be subject to stri

In [29]:
import PyPDF2

In [30]:
def parse_pdf(file_path):
    """
    Extracts text from a PDF document.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        str: Extracted text from the PDF document.
    """
    pdf_file = open(file_path, 'rb')
    text = []
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pages = pdf_reader.pages
        temp_text = ''
        for i, page_obj in enumerate(pages):
            if (num_tokens_from_string(temp_text) > 8150):
                text.append(temp_text)
                temp_text=''
            temp_text += page_obj.extract_text()
    return text

In [31]:
# Example usage:
file_path = './data/AI_ACT_2024.pdf'
extracted_text_list = parse_pdf(file_path)

In [32]:
agent = reasoning_engines.LangchainAgent(
    model=model,                # Required.
    model_kwargs=model_kwargs
)



In [ ]:
import time
refined_text = []
for extracted_text in extracted_text_list:
    prompt = "Repair the content of the text below. Make sure to not add any additional text other than what is already there as this would adulterate the content of the page. Only fix spacing and line issues. Do not hallucinate words into the content.\n\n {}".format(extracted_text)
    response = agent.query(input=prompt)
    time.sleep(45)
    refined_text.append(response['output'])

In [ ]:
output_text = ''
for text in refined_text:
    output_text += "{}\n\n".format(text)

with open('file.txt', 'w') as f:
    f.write(output_text)

In [ ]:
x = num_tokens_from_string(extracted_text)

In [ ]:
print(x)

In [ ]:
with open('file.txt', 'w') as f:
    f.write(response['output'])

In [ ]:
def get_tag_label(str):
    idx = str.find(']')
    return (str[1:idx], idx)

In [ ]:
def get_tag_content(start, str):
    end_idx = str.find('[', start)
    return str[start+1:end_idx].strip()

In [ ]:
CHAPTER = 'chapter'
CHAPTER_TITLE = 'chapter_title'
ARTICLE = 'article'
ARTICLE_TITLE = 'article_title'
ARTICLE_TEXT = 'article_text'
SECTION = 'section'
SECTION_TITLE = 'section_title'

In [ ]:
def get_data_fields_regulations(chapters, sections, articles):
    def get_details_from_regulations(f):
        chapter = None
        section = None
        article = None
        for line in f.readlines():
            (tag, start_idx) = get_tag_label(line)
            content = get_tag_content(start_idx, line)
            if (tag == CHAPTER):
                if(section != None):
                    sections.append(section)
                if (article != None):
                    articles.append(article)
                if(chapter != None):
                    chapters.append(chapter)
                    section = None
                    article = None
                chapter = {
                    "chapter_number": content[8:],
                }
            if (tag == CHAPTER_TITLE):
                chapter["chapter_title"] = content
            if (tag == SECTION):
                if (article != None):
                    articles.append(article)
                if(section != None):
                    article = None
                    sections.append(section)
                section = {
                    "chapter_number": chapter["chapter_number"],
                    "section_number": content[8:],
                }
            if (tag == SECTION_TITLE):
                section["section_title"] = content
            if (tag == ARTICLE):
                if(article != None):
                    articles.append(article)
                article = {
                    "article_number": content[8:],
                    "section_number": section["section_number"] if section != None else None,
                    "chapter_number": chapter["chapter_number"] if chapter != None else None,
                }
            if (tag == ARTICLE_TITLE):
                article['article_title'] = content
            if (tag == ARTICLE_TEXT):
                article['article_text'] = content
    return get_details_from_regulations

In [ ]:
def get_data_fields_annexes(articles):
    def get_data_from_annex(f):
        article = None
        for line in f.readlines():
            (tag, start_idx) = get_tag_label(line)
            content = get_tag_content(start_idx, line)
            if (tag == ARTICLE):
                if(article != None):
                    articles.append(article)
                article = {
                    "article_number": content[6:],
                }
            if (tag == ARTICLE_TITLE):
                article['article_title'] = content
            if (tag == ARTICLE_TEXT):
                article['article_text'] = content
    return get_data_from_annex

In [ ]:
def get_data_fields_references(references):
    def get_data_from_references(f):
        for line in f.readlines():
            (tag, start_idx) = get_tag_label(line)
            content = get_tag_content(start_idx, line)
            references.append({
                "reference_number": tag,
                "reference_text": content
            })
    return get_data_from_references

In [ ]:
def get_data_fields_guides(guides):
    def get_data_from_guides(f):
        for line in f.readlines():
            (tag, start_idx) = get_tag_label(line)
            content = get_tag_content(start_idx, line)
            guides.append({
                "id": tag,
                "text": content
            })
    return get_data_from_guides

In [ ]:
def create_array_from_file( file_path, get_details ):
    with open(file_path, 'r', encoding='utf8') as f:
        get_details(f)
            

In [ ]:
chapters = []
sections = []
articles = []
get_details_from_regulations = get_data_fields_regulations(chapters, sections, articles)
create_array_from_file('./regulation.txt', get_details_from_regulations )

In [ ]:
annex_articles = []
get_details_from_annex = get_data_fields_annexes(articles=annex_articles)
create_array_from_file('./annex.txt', get_details_from_annex)

In [ ]:
references = []
get_details_from_reference = get_data_fields_references(references)
create_array_from_file('./references.txt', get_details_from_reference)

In [ ]:
guides = []
get_data_guides = get_data_fields_guides(guides)
create_array_from_file('./guide.txt', get_data_guides)

In [ ]:
chapter_df = pd.DataFrame(chapters)
section_df = pd.DataFrame(sections)
article_df = pd.DataFrame(articles)

In [ ]:
annex_articles_df = pd.DataFrame(annex_articles)

In [ ]:
references_df = pd.DataFrame(references)

In [ ]:
guides_df = pd.DataFrame(guides)

In [ ]:
chapter_df.to_csv('ai_act_chapters.csv', index=False)
section_df.to_csv('ai_act_sections.csv', index=False)
article_df.to_csv('ai_act_articles.csv', index=False)

In [ ]:
annex_articles_df.to_csv('ai_act_annex.csv', index=False)

In [ ]:
references_df.to_csv('ai_act_references.csv', index=False)

In [ ]:
guides_df.to_csv('ai_act_guide.csv', index=False)

In [ ]:
merge_chapters_sections = pd.merge(chapter_df, section_df, how="left")

In [ ]:
merge_chapter_sections_articles = pd.merge(merge_chapters_sections, article_df, how="left")

In [ ]:
merge_chapter_sections_articles.head()

In [ ]:
merge_chapter_sections_articles.to_csv('ai_act_regulations.csv', index=False)